# Hugging Face Transformers 
Adapté de : cours NLP dispensé à Stanforfd et Hugging Face (Cours NLP)

Ce notebook présente la bibliothèque Python Hugging Face Transformers et quelques modèles courants que vous pouvez utiliser pour en tirer parti. Elle est particulièrement utile pour utiliser ou affiner des modèles de transformateurs pré-entraînés pour vos projets.

Hugging Face fournit un accès aux modèles (à la fois le code qui les implémente et leurs poids pré-entraînés (checkpoints), y compris les derniers LLMs comme Llama3, DBRX, etc), aux tokenizers spécifiques aux modèles, ainsi qu'aux pipelines pour les tâches courantes de NLP, et aux ensembles de données et métriques dans un paquetage séparé `datasets`. 
Il existe des implémentations en PyTorch, Tensorflow et Flax (mais nous utiliserons ici les versions PyTorch !).

Nous allons passer en revue quelques cas d'utilisation :
* Partie 1: Que peuvent faire les tansformers: Quelques Pipelines (end-to-end)(prêt à l'emploi - **off-the-shelf**) pour réaliser différentes tâches
* Partie 2 : Exploitation des transformers : vue d'ensemble des tokenizers et des modèles
* Partie 3 : Finetuning - pour votre propre tâche. Nous utiliserons un exemple de classification de sentiments.

Voici des ressources supplémentaires introduisant la bibliothèque qui ont été utilisées pour réaliser ce tutoriel :

* [Hugging Face Docs] (https://huggingface.co/docs/transformers/index)
  * Documentation claire
  * Tutoriels, visites guidées et carnets d'exemples
  * Liste des modèles disponibles
* [Hugging Face Course](https://huggingface.co/course/) (Plusieurs langues) 
* [Hugging Face Examples](https://github.com/huggingface/transformers/tree/main/examples/pytorch) Vous pouvez trouver des structures de code très similaires pour des tâches/modèles en aval très différents en utilisant Huggingface.


#  Partie I- Pipelines : que peuvent faire les *transformers* ?

🤗  proposent plusieurs pipelines, des modèles pré-entrainées pour réaliser des tâches spécifiques: analyse de sentiment, résumé, quesion réponse, NER, traduction.
(https://huggingface.co/docs/transformers/main_classes/pipelines)


Installez la bibliothèque 🤗 *Transformers* pour exécuter ce *notebook*.


In [ ]:
!pip install transformers -q
#!pip install transformers [sentencepiece] # pTransformers avec les dépendances pour SentencePiece.


In [ ]:
#!pip install tf-keras

### Les pipelines :
L’outil le plus basique de la bibliothèque 🤗 Transformers est la fonction pipeline(). Elle relie un modèle avec ses étapes de pré-traitement et de post-traitement, permettant d’entrer n’importe quel texte et d’obtenir une réponse compréhensible :
<img src="./images/pipline.jpg"  style="width:550px;height:150px;" >

In [ ]:
from transformers import pipeline

### 1 Exemple de pipeline : Analyse de sentiments

In [ ]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

In [ ]:
# Création de la pipeline
classifier = pipeline("sentiment-analysis")

# Inspection du modèle utilisé par défaut
print("Modèle utilisé : par défaut", classifier.model.name_or_path)


In [ ]:
import webbrowser
from transformers import pipeline

model_name = pipeline("sentiment-analysis").model.name_or_path
webbrowser.open(f"https://huggingface.co/{model_name}")


In [ ]:
classifier = pipeline("sentiment-analysis", model="tblard/tf-allocine")
classifier("J'ai attendu un cours d'HuggingFace toute ma vie.") 

Intéressant ! On observe que le résultat est négatif là où pour la version en anglais le résultat est positif.

#### On peut fournir plusieurs textes

In [ ]:
classifier(
    ["J'ai attendu un cours d'HuggingFace toute ma vie.", 
     "Je déteste tellement ça !"]
) # pour classifier plusieurs phrases

#### Voici une liste non-exhaustive des pipelines disponibles :

- feature-extraction (pour obtenir la représentation vectorielle d’un texte)
- fill-mask
- ner (named entity recognition ou reconnaissance d’entités nommées en français)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification

### 2.  Classification
#### Zéro shot classification
Classer des textes qui n’ont pas été annotés. 

In [ ]:
classifier = pipeline("zero-shot-classification")
classifier(
    "C'est un cours sur la bibliothèque Transformers",
    candidate_labels=["éducation", "politique", "affaires"],
)

### 3.  Génération de textes

In [ ]:
generator = pipeline("text-generation", model="asi/gpt-fr-cased-small")
generator("# Dans ce cours, nous vous enseignerons comment")

In [ ]:
# version en anglais
enerator = pipeline("text-generation")
generator("In this course, we will teach you how to")

In [ ]:
# model="distilgpt2", ou un autre modèle pour les textes en anglais
#model="asi/gpt-fr-cased-small" pour des yextes en français (exemple pris à ttre indicatif)
generator = pipeline("text-generation",model="distilgpt2" )
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=1,
)

### 4. Remplacement des mots masqués

In [ ]:
#exemple de modèles
#model="camembert-base" pour le francais
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

### 5. Reconnaissance d'entités nommées

In [ ]:
#modèles
# pour le rancais : model="Jean-Baptiste/camembert-ner"
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")
#ner("Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.")

### 6. Réponse à des questions

In [ ]:
#Modèle pour le français
#model="etalab-ia/camembert-base-squadFR-fquad-piaf"
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

In [ ]:

# Inspection du modèle utilisé par défaut
print("Modèle utilisé :", question_answerer.model.name_or_path)


### 7. Résumé

In [ ]:
summarizer = pipeline("summarization", model="moussaKam/barthez-orangesum-abstract")
summarizer(
    """
    L'Amérique a changé de façon spectaculaire au cours des dernières années. Non seulement le nombre de 
    diplômés dans les disciplines traditionnelles de l'ingénierie telles que le génie mécanique, civil, 
    l'électricité, la chimie et l'aéronautique a diminué, mais dans la plupart 
    des grandes universités américaines, les programmes d'études d'ingénierie se concentrent désormais sur 
    et encouragent largement l'étude des sciences de l'ingénieur. Par conséquent, il y a 
    de moins en moins d'offres dans les sujets d'ingénierie traitant de l'infrastructure, 
    l'environnement et les questions connexes, et une plus grande concentration sur les sujets de haute 
    technologie, qui soutiennent en grande partie des développements scientifiques de plus en plus 
    complexes. Si cette dernière est importante, elle ne doit pas se faire au détriment
    de l'ingénierie plus traditionnelle.

    Les économies en développement rapide telles que la Chine et l'Inde, ainsi que d'autres 
    pays industrialisés d'Europe et d'Asie, continuent d'encourager et de promouvoir
    l'enseignement de l'ingénierie. La Chine et l'Inde, respectivement, diplôment 
    six et huit fois plus d'ingénieurs traditionnels que les États-Unis. 
    Les autres pays industriels maintiennent au minimum leur production, tandis que l'Amérique 
    souffre d'une baisse de plus en plus importante du nombre de diplômés en ingénierie
    et un manque d'ingénieurs bien formés.
"""
)

### 9. Traduction

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")
translator("This course is produced by Hugging Face.")

### 10. Pipleone avec choix de la tâche et du modèle

In [ ]:
from transformers import pipeline

mon_pipeline = pipeline(
    task="nom_de_la_tâche",
    model="nom_du_modèle",
    tokenizer="nom_du_tokenizer",  # optionnel si identique au modèle
)


In [ ]:
from transformers import pipeline

classifier = pipeline(
    task="sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

resultats = classifier("Ce film est excellent !")
print(resultats)


| Tâche (`task`)            | Description                                      |
|--------------------------|--------------------------------------------------|
| `sentiment-analysis`     | Classification de texte (binaire ou multi-classes) |
| `text-classification`    | Synonyme générique de `sentiment-analysis`       |
| `translation`            | Traduction automatique                           |
| `summarization`          | Résumé automatique de texte                      |
| `question-answering`     | Réponse à une question sur un contexte donné     |
| `text-generation`        | Génération de texte (type GPT)                   |
| `fill-mask`              | Prédiction de mot masqué (type BERT)            |
| `ner`                    | Reconnaissance d'entités nommées (Named Entity Recognition) |
| `zero-shot-classification` | Classification sans fine-tuning spécifique
